**<h2> Embedded Systems Course Project </h2>**
<hr>

**<h3> Title : DL model optimization for
Lightweight Visual Wake Words
Task</h3>**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir "/content/drive/MyDrive/ES_Model_weights"

## Data Loading and Processing

In [ ]:
!pip install pyvww

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/Mxbonn/visualwakewords.git

Cloning into 'visualwakewords'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 79 (delta 1), reused 6 (delta 1), pack-reused 71
Unpacking objects: 100% (79/79), 892.43 KiB | 2.48 MiB/s, done.


In [ ]:
%cd '/content/visualwakewords'

/content/visualwakewords


In [ ]:
!bash scripts/download_mscoco.sh '/content/coco' 2014

--2023-05-05 13:42:57--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.89.220, 3.5.6.131, 52.216.10.115, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.89.220|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  51.4MB/s    in 3m 38s  

2023-05-05 13:46:35 (59.2 MB/s) - ‘train2014.zip’ saved [13510573713/13510573713]

Unzipping train2014.zip
--2023-05-05 13:50:17--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.165.97, 52.217.139.201, 3.5.28.38, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.165.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[====

In [ ]:
%cd ..

/content


In [ ]:
TRAIN_ANNOTATIONS_FILE="/content/coco/annotations/instances_train2014.json"
VAL_ANNOTATIONS_FILE="/content/coco/annotations/instances_val2014.json"
DIR="/content/coco/annotations"
!python visualwakewords/scripts/create_coco_train_minival_split.py \
  --train_annotations_file=$TRAIN_ANNOTATIONS_FILE \
  --val_annotations_file=$VAL_ANNOTATIONS_FILE \
--output_dir=$DIR

In [ ]:
MAXITRAIN_ANNOTATIONS_FILE="/content/coco/annotations/instances_maxitrain.json"
MINIVAL_ANNOTATIONS_FILE="/content/coco/annotations/instances_minival.json"
VWW_OUTPUT_DIR="/content/dataset/annotations"
!python /content/visualwakewords/scripts/create_visualwakewords_annotations.py \
  --train_annotations_file=$MAXITRAIN_ANNOTATIONS_FILE \
  --val_annotations_file=$MINIVAL_ANNOTATIONS_FILE \
  --output_dir=$VWW_OUTPUT_DIR \
  --threshold=0.005 \
  --foreground_class='person'

Processing /content/coco/annotations/instances_maxitrain.json...
loading annotations into memory...
Done (t=22.18s)
creating index...
index created!
There are 55233 images that now have label person, of the 115228 images in total.
Processing /content/coco/annotations/instances_minival.json...
loading annotations into memory...
Done (t=0.86s)
creating index...
index created!
There are 3800 images that now have label person, of the 8059 images in total.


In [ ]:
from torch.utils.data import random_split
import torch
import pyvww
import torchvision
import torchvision.transforms as transforms

In [ ]:
train_dataset = pyvww.pytorch.VisualWakeWordsClassification(root="/content/coco/all2014", 
                    annFile="/content/dataset/annotations/instances_train.json")
# print(len(train_dataset))
val_dataset = pyvww.pytorch.VisualWakeWordsClassification(root="/content/coco/all2014", 
                    annFile="/content/dataset/annotations/instances_val.json")

loading annotations into memory...
Done (t=5.20s)
creating index...
index created!
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


In [ ]:
print(train_dataset)

Dataset VisualWakeWordsClassification
    Number of datapoints: 115228
    Root location: /content/coco/all2014


In [ ]:
print(val_dataset)

Dataset VisualWakeWordsClassification
    Number of datapoints: 8059
    Root location: /content/coco/all2014


In [ ]:
train_dataset[0][0].size

(640, 480)

In [ ]:
import torch
import pyvww
import torchvision
import torchvision.transforms as transforms
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = pyvww.pytorch.VisualWakeWordsClassification(root="/content/coco/all2014", 
                    annFile="/content/dataset/annotations/instances_train.json", transform=train_transform)
# print(len(train_dataset))
val_dataset = pyvww.pytorch.VisualWakeWordsClassification(root="/content/coco/all2014", 
                    annFile="/content/dataset/annotations/instances_val.json", transform=test_transform)


loading annotations into memory...
Done (t=6.93s)
creating index...
index created!
loading annotations into memory...
Done (t=1.49s)
creating index...
index created!


In [ ]:
print(train_dataset)

Dataset VisualWakeWordsClassification
    Number of datapoints: 115228
    Root location: /content/coco/all2014
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [ ]:
print(val_dataset)

Dataset VisualWakeWordsClassification
    Number of datapoints: 8059
    Root location: /content/coco/all2014
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )


In [ ]:
train_dataset[0][0].shape

torch.Size([3, 128, 128])

In [ ]:
from torch.utils.data import DataLoader
batch_size = 64
val_batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_size = int(0.5 * len(val_dataset))
test_size = len(val_dataset) -val_size
print("val_size = ",val_size)
print("test_size = ",test_size)
val_dataset, test_dataset = random_split(val_dataset, [val_size,test_size])
val_dataloader = DataLoader(val_dataset, batch_size=val_batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=val_batch_size, shuffle=True)

val_size =  4029
test_size =  4030


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm

In [ ]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchsummary import summary

## Model 1
2 Conv layer and 3 FC layers

### Training

In [ ]:
import torch.nn as nn

class classifier1(nn.Module):
    def __init__(self):
        super(classifier1, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

model = classifier1().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
max_val_acc = 0
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    step = 0
    for i, data in tqdm(enumerate(train_dataloader)):
        step+=1
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        # break
        # torch.save(model.state_dict(),"/content/model1.pt")
    with torch.no_grad():
        model.eval()
        acc = 0
        for idx, (images,labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            acc+= torch.sum(predicted==labels)
        # print(acc, val_size)
        print(f'Validation accuracy: {(100*acc)/val_size} %')
        if acc > max_val_acc:
            max_val_acc = acc
            torch.save(model.state_dict(),"/content/model1.pt")
    print('Epoch %d, Loss: %.3f' % (epoch+1, running_loss / len(train_dataloader)))

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validation accuracy: 64.33358001708984 %
Epoch 1, Loss: 0.614


0it [00:00, ?it/s]

Validation accuracy: 69.79399871826172 %
Epoch 2, Loss: 0.550


...

Validation accuracy: 72.64830017089844 %
Epoch 3, Loss: 0.511


0it [00:00, ?it/s]

Validation accuracy: 73.71556091308594 %
Epoch 4, Loss: 0.472


0it [00:00, ?it/s]

Validation accuracy: 73.36808013916016 %
Epoch 5, Loss: 0.432


In [ ]:
!cp "/content/model1.pt" "/content/drive/MyDrive/ES_Model_weights/model1.pt"
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                  [-1, 128]       3,686,528
            Linear-6                   [-1, 64]           8,256
            Linear-7                    [-1, 2]             130
Total params: 3,700,002
Trainable params: 3,700,002
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 14.11
Estimated Total Size (MB): 17.85
----------------------------------------------------------------


### Inference of Model 1

In [ ]:

class classifier1(nn.Module):
    def __init__(self):
        super(classifier1, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)
        return x

In [ ]:
import torch.nn.functional as F
# del model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = classifier1().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model1.pt",map_location = device))
test_acc = 0
model.eval()

import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 74.98759460449219 %


In [ ]:
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                  [-1, 128]       3,686,528
            Linear-6                   [-1, 64]           8,256
            Linear-7                    [-1, 2]             130
Total params: 3,700,002
Trainable params: 3,700,002
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 14.11
Estimated Total Size (MB): 17.85
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/test_size} seconds per test sample")

Total time taken for infernce is:37.760234117507935 seconds or 0.018739570281641655 seconds per test sample


### Inference with quantization

In [ ]:
class MyCNN1(nn.Module):
    def __init__(self):
        super(MyCNN1, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)
        return x

In [ ]:
import torch.nn.functional as F
test_acc = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MyCNN1().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model1.pt",map_location=device))

model.eval()

quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8
)

import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = quantized_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 74.88833618164062 %


In [ ]:
summary(quantized_model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                  [-1, 128]               0
            Linear-6                   [-1, 64]               0
            Linear-7                    [-1, 2]               0
Total params: 5,088
Trainable params: 5,088
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 0.02
Estimated Total Size (MB): 3.76
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/(test_size)} seconds per test sample")

Total time taken for infernce is:38.499837160110474 seconds or 0.019106618938020088 seconds per test sample


In [ ]:
del model

## Model 2 

### Training

In [ ]:
import torch.nn as nn
class MyCNN2(nn.Module):
    def __init__(self):
        super(MyCNN2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate your CNN object and define the loss and optimizer
model = MyCNN2().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train your CNN
num_epochs = 5
max_val_acc = 0
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    step = 0
    for i, data in tqdm(enumerate(train_dataloader)):
        step+=1
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    with torch.no_grad():
        model.eval()
        acc = 0
        for idx, (images,labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            acc+= torch.sum(predicted==labels)
        print(f'Validation accuracy: {(100*acc)/val_size} %')
        if acc > max_val_acc:
            max_val_acc = acc
            torch.save(model.state_dict(),"/content/model2.pt")
    print('Epoch %d, Loss: %.3f' % (epoch+1, running_loss / len(train_dataloader)))

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validation accuracy: 68.05659484863281 %
Epoch 1, Loss: 0.612


0it [00:00, ?it/s]

Validation accuracy: 70.43931579589844 %
Epoch 2, Loss: 0.545


...

Validation accuracy: 72.49938201904297 %
Epoch 3, Loss: 0.505


0it [00:00, ?it/s]

Validation accuracy: 73.8148422241211 %
Epoch 4, Loss: 0.471


0it [00:00, ?it/s]

Validation accuracy: 74.43534851074219 %
Epoch 5, Loss: 0.434


In [ ]:
!cp "/content/model2.pt" "/content/drive/MyDrive/ES_Model_weights/model2.pt"
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                  [-1, 128]       3,686,528
            Linear-6                    [-1, 2]             258
Total params: 3,691,874
Trainable params: 3,691,874
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 14.08
Estimated Total Size (MB): 17.82
----------------------------------------------------------------


### Inference of Model 2

In [ ]:
class MyCNN2(nn.Module):
    def __init__(self):
        super(MyCNN2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

In [ ]:
import torch.nn.functional as F
# del model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyCNN2().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model2.pt",map_location = device))
test_acc = 0
model.eval()

import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 76.22828674316406 %


In [ ]:
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                  [-1, 128]       3,686,528
            Linear-6                    [-1, 2]             258
Total params: 3,691,874
Trainable params: 3,691,874
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 14.08
Estimated Total Size (MB): 17.82
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/test_size} seconds per test sample")

Total time taken for infernce is:37.54535174369812 seconds or 0.018632928905061102 seconds per test sample


### Inference with Quantization

In [ ]:
class MyCNN2(nn.Module):
    def __init__(self):
        super(MyCNN2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

In [ ]:
import torch.nn.functional as F
# del model
model = MyCNN2().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model2.pt",map_location = device))
test_acc = 0
model.eval()
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8
)
import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = quantized_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 76.32754516601562 %


In [ ]:
summary(quantized_model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                  [-1, 128]               0
            Linear-6                    [-1, 2]               0
Total params: 5,088
Trainable params: 5,088
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 0.02
Estimated Total Size (MB): 3.76
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/(test_size)} seconds per test sample")

Total time taken for infernce is:39.14583158493042 seconds or 0.01942721170468011 seconds per test sample


In [ ]:
del model

## Model 3

### Training

In [ ]:
import torch.nn as nn

class MyCNN3(nn.Module):
    def __init__(self):
        super(MyCNN3, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate your CNN object and define the loss and optimizer
model = MyCNN3().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train your CNN
num_epochs = 5
max_val_acc = 0
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    step = 0
    for i, data in tqdm(enumerate(train_dataloader)):
        step+=1
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    with torch.no_grad():
        model.eval()
        acc = 0
        for idx, (images,labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            acc+= torch.sum(predicted==labels)
        print(f'Validation accuracy: {(100*acc)/val_size} %')
        if acc > max_val_acc:
            max_val_acc = acc
            torch.save(model.state_dict(),"/content/model3.pt")
    print('Epoch %d, Loss: %.3f' % (epoch+1, running_loss / len(train_dataloader)))

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validation accuracy: 68.4537124633789 %
Epoch 1, Loss: 0.627


0it [00:00, ?it/s]

Validation accuracy: 71.55622100830078 %
Epoch 2, Loss: 0.558


...

Validation accuracy: 72.62348175048828 %
Epoch 3, Loss: 0.528


0it [00:00, ?it/s]

Validation accuracy: 73.64110565185547 %
Epoch 4, Loss: 0.504


0it [00:00, ?it/s]

Validation accuracy: 72.3504638671875 %
Epoch 5, Loss: 0.484


In [ ]:
!cp "/content/model3.pt" "/content/drive/MyDrive/ES_Model_weights/model3.pt"
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                   [-1, 64]       1,843,264
            Linear-6                    [-1, 2]             130
Total params: 1,848,482
Trainable params: 1,848,482
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 7.05
Estimated Total Size (MB): 10.79
----------------------------------------------------------------


### Inference of Model 3

In [ ]:
class MyCNN3(nn.Module):
    def __init__(self):
        super(MyCNN3, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

In [ ]:
import torch.nn.functional as F
# del model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyCNN3().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model3.pt",map_location = device))
test_acc = 0
model.eval()

import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 73.07691955566406 %


In [ ]:
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                   [-1, 64]       1,843,264
            Linear-6                    [-1, 2]             130
Total params: 1,848,482
Trainable params: 1,848,482
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 7.05
Estimated Total Size (MB): 10.79
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/test_size} seconds per test sample")

Total time taken for infernce is:96.10018968582153 seconds or 0.023846200914595914 seconds per test sample


### Inference with Quantization

In [ ]:
class MyCNN3(nn.Module):
    def __init__(self):
        super(MyCNN3, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 30 * 30, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 30 * 30)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

In [ ]:
import torch.nn.functional as F
# del model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyCNN3().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model3.pt",map_location = device))
test_acc = 0
model.eval()
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8
)
import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = quantized_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 73.10173797607422 %


In [ ]:
summary(quantized_model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Conv2d-3           [-1, 32, 61, 61]           4,640
         MaxPool2d-4           [-1, 32, 30, 30]               0
            Linear-5                   [-1, 64]               0
            Linear-6                    [-1, 2]               0
Total params: 5,088
Trainable params: 5,088
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 3.55
Params size (MB): 0.02
Estimated Total Size (MB): 3.76
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/test_size} seconds per test sample")

Total time taken for infernce is:85.05028915405273 seconds or 0.02110429011266817 seconds per test sample


## Model 4

### Training

In [ ]:
import torch.nn as nn

class MyCNN4(nn.Module):
    def __init__(self):
        super(MyCNN4, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        # self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(16 *63*63, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        # print(x.shape)
        x = self.pool(x)
        # print(x.shape)
        # x = self.conv2(x)
        # x = nn.ReLU()(x)
        # x = self.pool(x)
        x = x.view(-1, 16 * 63 * 63)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

# Instantiate your CNN object and define the loss and optimizer
model = MyCNN4().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train your CNN
num_epochs = 5
max_val_acc = 0
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0.0
    step = 0
    for i, data in tqdm(enumerate(train_dataloader)):
        step+=1
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    with torch.no_grad():
        model.eval()
        acc = 0
        for idx, (images,labels) in enumerate(val_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            acc+= torch.sum(predicted==labels)
        print(f'Validation accuracy: {(100*acc)/val_size} %')
        if acc > max_val_acc:
            max_val_acc = acc
            torch.save(model.state_dict(),"/content/model4.pt")
    print('Epoch %d, Loss: %.3f' % (epoch+1, running_loss / len(train_dataloader)))

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Validation accuracy: 62.79473876953125 %
Epoch 1, Loss: 0.636


0it [00:00, ?it/s]

Validation accuracy: 67.33680725097656 %
Epoch 2, Loss: 0.593


...

Validation accuracy: 66.89004516601562 %
Epoch 3, Loss: 0.569


0it [00:00, ?it/s]

Validation accuracy: 67.53536987304688 %
Epoch 4, Loss: 0.547


0it [00:00, ?it/s]

Validation accuracy: 67.63465118408203 %
Epoch 5, Loss: 0.528


In [ ]:
!cp "/content/model4.pt" "/content/drive/MyDrive/ES_Model_weights/model4.pt"
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Linear-3                   [-1, 64]       4,064,320
            Linear-4                    [-1, 2]             130
Total params: 4,064,898
Trainable params: 4,064,898
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 2.42
Params size (MB): 15.51
Estimated Total Size (MB): 18.12
----------------------------------------------------------------


### Inference of Model 4

In [ ]:
import torch.nn as nn

class MyCNN4(nn.Module):
    def __init__(self):
        super(MyCNN4, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        # self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(16 *63*63, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        # print(x.shape)
        x = self.pool(x)
        # print(x.shape)
        # x = self.conv2(x)
        # x = nn.ReLU()(x)
        # x = self.pool(x)
        x = x.view(-1, 16 * 63 * 63)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch.nn.functional as F
# del model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyCNN4().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model4.pt",map_location = device))
test_acc = 0
model.eval()

import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 68.33747100830078 %


In [ ]:
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Linear-3                   [-1, 64]       4,064,320
            Linear-4                    [-1, 2]             130
Total params: 4,064,898
Trainable params: 4,064,898
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 2.42
Params size (MB): 15.51
Estimated Total Size (MB): 18.12
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/test_size} seconds per test sample")

Total time taken for infernce is:85.16756057739258 seconds or 0.021133389721437364 seconds per test sample


### Inference with Quantization

In [ ]:
import torch.nn.functional as F
# del model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyCNN4().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/ES_Model_weights/model4.pt",map_location = device))
test_acc = 0
model.eval()
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8
)
import time
start = time.time()
with torch.no_grad():
    for idx, (images,labels) in enumerate(test_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = quantized_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_acc+=torch.sum(predicted==labels)
end = time.time()-start
print(f'testing accuracy: {(100*test_acc)/test_size} %')

testing accuracy: 68.26303100585938 %


In [ ]:
summary(quantized_model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 126, 126]             448
         MaxPool2d-2           [-1, 16, 63, 63]               0
            Linear-3                   [-1, 64]               0
            Linear-4                    [-1, 2]               0
Total params: 448
Trainable params: 448
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 2.42
Params size (MB): 0.00
Estimated Total Size (MB): 2.61
----------------------------------------------------------------


In [ ]:
print(f"Total time taken for infernce is:{end} seconds or {end/test_size} seconds per test sample")

Total time taken for infernce is:65.34195613861084 seconds or 0.01621388489791832 seconds per test sample
